<a href="https://colab.research.google.com/github/edilsonclodoalves/AnimaCursoExtensaoPython2022-2/blob/main/TerceiroExemploPandasDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dashboard de Crimes em São Paulo**

Requisitos são:
Criar uma conta no dataplane
https://cloud.datapane.com/accounts/login/?next=/home/

Criar uma conta no https://ngrok.com/

Pegar o token dos dois.
Procurar as seções # 4 - Tokern DataPane e # 19 - TOKEN NGROK e inserir os tokens respectivamente.

In [141]:
# 1 -
#INSTALAÇÃO DAS BIBLIOTECAS
# Install Plotly
!pip install Plotly==4.12

# Install Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

#Install datapane
!pip install datapane

#Install brokeh
!pip install bokeh~=2.4.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached plotly-4.12.0-py2.py3-none-any.whl (13.1 MB)
  Attempting uninstall: Plotly
    Found existing installation: plotly 5.13.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/cli/base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/cli/req_command.py", line 247, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/commands/install.py", line 494, in run
    installed = install_given_reqs(
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/req/__init__.py", line 68, in install_given_reqs
    uninstalled_pathset = requirement.uninstall(auto_confirm=True)
  File "/usr/local/lib/python3.10/site-packages/pip/_internal/req/req_install.py", line 659, in uninstall
    uninstalled_path

In [ ]:
# 2 - 
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda init bash
!conda config --set auto_activate_base False

In [86]:
# 3 -
import datapane as dp
import os.path
import sys, json
import requests
import subprocess

import numpy as np
import pandas as pd
import plotly.express as px

from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [87]:
# 4 - Tokern DataPane
dp.login(token="TOKEN_DO_DATAPANE")

Connected successfully to https://cloud.datapane.com as edilson.lima4931@aluno.una.br


'edilson.lima4931@aluno.una.br'

3.0 Limpando os dados
Serão utilizados apenas as colunas bairro, latitude, longitude, sexo e data.

In [139]:
# 5 -
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/dataset-limpo.csv')
dataset.head()

,id,bairro,created_at,descricao,endereco,id.1,ip_address_origin,latitude,longitude,registrou_bo,...,Estepe,MP4 ou Ipod,Móveis,Notebook,Outros,Relógio,Som,Tablet,Tv,time
0,49787,Butantã,2015-06-16 23:34:01.000,"5 bandidos, armas, entraram em casa e vasculha...","Avenida Trona Constanzo, 177, Butantã",49787,NaN,-23.580973,-46.719192,True,...,NaN,True,NaN,True,NaN,NaN,True,True,True,2010-12-20 20:30:00.000
1,10679,Itaquera,2013-09-11 18:26:31.819,"Meu caso foi para o Pequenas Causas, ganhamos ...","Avenida José Pinheiro Borges, 450-876 - Itaque...",10679,187.11.1.51,-23.540105,-46.462872,True,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2012-01-11 14:00:00.000
2,10682,Itaquera,2013-09-11 18:42:07.858,"Meu caso foi para o Pequenas Causas, ganhamos ...",Rua Corínthians - Itaquera - Shopping Metrô It...,10682,189.19.160.82,-23.541765,-46.469562,True,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,2012-01-11 14:00:00.000
3,6166,Morumbi,2013-08-20 11:56:52.717,"Estava parada no semáforo, o bandido veio e es...","Rua Pasquale Gallupi, 347 - Morumbi, São Paulo...",6166,199.67.140.46,-23.611693,-46.725935,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-01-11 18:20:00.000
4,6521,Alto de Pinheiros,2013-08-20 17:41:21.569,Arrombaram o porta malas do meu carro e levara...,"Avenida Professor Fonseca Rodrigues, 1655 - Al...",6521,187.92.158.154,-23.544564,-46.721567,True,...,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,2012-01-11 08:15:00.000


In [140]:
# 6 -
crime = dataset.iloc[:, [1,7,8,10,35]]
crime.head()

,bairro,latitude,longitude,sexo,time
0,Butantã,-23.580973,-46.719192,0,2010-12-20 20:30:00.000
1,Itaquera,-23.540105,-46.462872,1,2012-01-11 14:00:00.000
2,Itaquera,-23.541765,-46.469562,1,2012-01-11 14:00:00.000
3,Morumbi,-23.611693,-46.725935,0,2012-01-11 18:20:00.000
4,Alto de Pinheiros,-23.544564,-46.721567,1,2012-01-11 08:15:00.000


In [97]:
# 7 -
crime["hora"] = pd.to_datetime(crime.time.str.rsplit(n=1, expand=True)[1].str[:-4])
crime["aux"] = (crime.time.str.rsplit(n=1, expand=True)[1].str[:-10]).astype(int)

<ipython-input-97-f358882dff39>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-97-f358882dff39>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
# 8 -
#Arredondando as horas
crime['hora'] = crime.hora.dt.floor('h').dt.strftime('%H:%M')

<ipython-input-98-4e198b0605ab>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [99]:
# 9 -
crime["time"] = pd.to_datetime(crime.time.str.rsplit(n=1, expand=True)[0])
crime = crime.rename(columns={"time": "data"})
crime["dia_da_semana"] = crime.data.dt.dayofweek

<ipython-input-99-a21d307ed14b>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [100]:
# 10 -
crime.head()

,bairro,latitude,longitude,sexo,data,hora,aux,dia_da_semana
0,Butantã,-23.580973,-46.719192,0,2010-12-20,20:00,20,0
1,Itaquera,-23.540105,-46.462872,1,2012-01-11,14:00,14,2
2,Itaquera,-23.541765,-46.469562,1,2012-01-11,14:00,14,2
3,Morumbi,-23.611693,-46.725935,0,2012-01-11,18:00,18,2
4,Alto de Pinheiros,-23.544564,-46.721567,1,2012-01-11,08:00,8,2


In [101]:
# 11 -
crime.hora.unique().shape

(24,)

In [102]:
# 12 -
crime_bar = crime[['hora','dia_da_semana', 'aux']].groupby(['hora', 'aux']).count()
crime_bar.head()

,,dia_da_semana
hora,aux,
00:00,0,302
01:00,1,214
02:00,2,205
03:00,3,168
04:00,4,185


In [103]:
# 13 -
crime_bar.rename(columns={'dia_da_semana':'Soma'}, inplace=True)
crime_bar.head()

,,Soma
hora,aux,
00:00,0,302
01:00,1,214
02:00,2,205
03:00,3,168
04:00,4,185


In [105]:
# 14 - Bar graph here
fig = px.bar(crime_bar, 
             x=crime_bar.index.get_level_values('hora'), 
             y="Soma",
             text="Soma", 
             labels={'x':'Hora do dia'},
             color=np.arange(0, 24, 1).tolist(), 
             color_continuous_scale=px.colors.sequential.Viridis_r,
             opacity=1,
             title="Número de crimes por hora do dia em São Paulo")
          

fig.update_layout(
    title_font_size=20, 
    titlefont=dict(color='white'))
fig.update_traces(
    texttemplate='%{text:.2s}', 
    textposition='outside',
    textfont=dict(color='white'))
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_yaxes(title="Crimes", titlefont=dict(color='white'), tickfont=dict(color='white'))
fig.update_xaxes(titlefont=dict(color='white'), tickfont=dict(color='white'))

fig.update_layout(coloraxis_showscale=False)
fig.update_layout({
'plot_bgcolor': '#323130',
'paper_bgcolor': '#323130',
})
report = dp.Report(dp.Plot(fig) ) #Create a report
report.upload(name='crime_bar', open=True, visibility='PUBLIC') #Publish the report
fig.show()

Uploading app and associated data - *please wait...*

App successfully uploaded. View and share your app at <a href='https://cloud.datapane.com/apps/0kevmJA/crime-bar/' target='_blank'>here</a>.

In [106]:
# 15 -
crime_map = crime[['hora','latitude', 'longitude','dia_da_semana', 'aux' ]].groupby(['hora','latitude', 'longitude', 'aux']).count()
crime_map.head()

dia_da_semana
hora  latitude   longitude  aux               
00:00 -23.829716 -46.720627 0                1
      -23.827724 -46.730026 0                1
      -23.813213 -46.709600 0                1
      -23.776135 -46.688304 0                1
      -23.759459 -46.667662 0                1

In [107]:
# 16 -
# transforms multi-index in columns
crime_map.reset_index(level=['hora','latitude', 'longitude', 'aux'],inplace=True)
crime_map.rename(columns={'dia_da_semana':'Crimes'}, inplace=True)

crime_map.head()

,hora,latitude,longitude,aux,Crimes
0,00:00,-23.829716,-46.720627,0,1
1,00:00,-23.827724,-46.730026,0,1
2,00:00,-23.813213,-46.709600,0,1
3,00:00,-23.776135,-46.688304,0,1
4,00:00,-23.759459,-46.667662,0,1


In [121]:
# 17 -
token = 'pk.eyJ1IjoiYW5kcmVoZ3VzdGF2byIsImEiOiJja2hyemc0MnkwODZoMnBtdGIzeGNqdG1sIn0.JJmZaJRtTutT5X24KS-4pw'
fig = px.scatter_mapbox(crime_map, 
                        lat='latitude', 
                        lon='longitude',
                        size_max=40, 
                        zoom=10,
                        labels={'aux':'Hora'},                        
                        color='aux',                        
                        color_continuous_scale=px.colors.sequential.Viridis_r,                          
                        mapbox_style='carto-positron')
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.update_layout(coloraxis_colorbar=dict(
    title="Horário do dia",
    bgcolor= None,   
    dtick=1,
    tickfont=dict(color='white'),
    titlefont=dict(color='white'),
))
fig.update_layout({
'plot_bgcolor': '#323130',
'paper_bgcolor': '#323130',
})
report = dp.Report(dp.Plot(fig) ) #Create a report
report.upload(name='crime_map', open=True, visibility='PUBLIC') #Publish the report
fig.show()

Uploading app and associated data - *please wait...*

App successfully uploaded. View and share your app at <a href='https://cloud.datapane.com/apps/n3ZxQGk/crime-map/' target='_blank'>here</a>.

In [ ]:
# 18 -
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


In [ ]:
# 19 - TOKEN NGROK
!./ngrok authtoken <TOKEN_DO_NGROK>

In [131]:
Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [132]:
# 20 -
%%writefile my_dash_app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('dataset-limpo.csv')
crime = df.iloc[:, [1,7,8,10,35]]
crime["hora"] = pd.to_datetime(crime.time.str.rsplit(n=1, expand=True)[1].str[:10])
crime["aux"] = (crime.time.str.rsplit(n=1, expand=True)[1].str[:-10]).astype(int)
crime['hora'] = crime.hora.dt.floor('h').dt.strftime('%H:%M')
crime["time"] = pd.to_datetime(crime.time.str.rsplit(n=1, expand=True)[0])
crime = crime.rename(columns={"time": "data"})
crime["dia_da_semana"] = crime.data.dt.dayofweek
crime_map = crime[['hora','latitude', 'longitude','dia_da_semana', 'aux' ]].groupby(['hora','latitude', 'longitude', 'aux']).count()
crime_map.head()
crime_map.reset_index(level=['hora','latitude', 'longitude', 'aux'],inplace=True)
crime_map.rename(columns={'dia_da_semana':'Crimes'}, inplace=True)

token = 'pk.eyJ1IjoiYW5kcmVoZ3VzdGF2byIsImEiOiJja2hyemc0MnkwODZoMnBtdGIzeGNqdG1sIn0.JJmZaJRtTutT5X24KS-4pw'
fig = px.scatter_mapbox(crime_map, 
                        lat='latitude', 
                        lon='longitude',
                        size_max=40, 
                        zoom=10,                      
                        color='aux',
                        labels={'aux':'Hora'},
                        color_continuous_scale=px.colors.sequential.Viridis_r,                          
                        mapbox_style='carto-positron')
fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig.update_layout(coloraxis_colorbar=dict(
    title="Horário do dia",
    bgcolor= None,   
    dtick=1,
    tickfont=dict(color='white'),
    titlefont=dict(color='white'),
))
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

crime_bar = crime[['hora','dia_da_semana', 'aux']].groupby(['hora', 'aux']).count()
crime_bar.rename(columns={'dia_da_semana':'Soma'}, inplace=True)

fig2 = px.bar(crime_bar, 
             x=crime_bar.index.get_level_values('hora'), 
             y="Soma",
             text="Soma", 
             labels={'x':'Hora do dia'},
             color=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23], color_continuous_scale=px.colors.sequential.Viridis_r,
             barmode="overlay",
             opacity=1,
             title="Número de crimes por hora do dia em São Paulo")
          

fig2.update_layout(
    title_font_size=20, 
    titlefont=dict(color='white'))
fig2.update_traces(
    texttemplate='%{text:.2s}', 
    textposition='outside',
    textfont=dict(color='white'))
fig2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig2.update_yaxes(title="Crimes", titlefont=dict(color='white'), tickfont=dict(color='white'))
fig2.update_xaxes(titlefont=dict(color='white'), tickfont=dict(color='white'))

fig2.update_layout(coloraxis_showscale=False)
fig2.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})



# Layout do Dash
app.layout = html.Div([                      

    html.H1("Crimes por Hora em São Paulo", style={'text-align':'center', 'color':'white'}),
    
    dcc.Graph(
        id='crime-map',
        figure=fig
    ),
    dcc.Dropdown(
               id='bar-selector',
               options=[
                   {'label': '0:00', 'value': '0'},
                   {'label': '1:00', 'value': '1'},
                   {'label': '2:00', 'value': '2'},
                   {'label': '3:00', 'value': '3'},
                   {'label': '4:00', 'value': '4'},
                   {'label': '5:00', 'value': '5'},
                   {'label': '6:00', 'value': '6'},
                   {'label': '7:00', 'value': '7'},
                   {'label': '8:00', 'value': '8'},
                   {'label': '9:00', 'value': '9'},
                   {'label': '10:00', 'value': '10'},
                   {'label': '11:00', 'value': '11'},
                   {'label': '12:00', 'value': '12'},
                   {'label': '13:00', 'value': '13'},
                   {'label': '14:00', 'value': '14'},
                   {'label': '15:00', 'value': '15'},
                   {'label': '16:00', 'value': '16'},
                   {'label': '17:00', 'value': '17'},
                   {'label': '18:00', 'value': '18'},
                   {'label': '19:00', 'value': '19'},
                   {'label': '20:00', 'value': '20'},
                   {'label': '21:00', 'value': '21'},
                   {'label': '22:00', 'value': '22'},
                   {'label': '23:00', 'value': '23'}
               ],
               multi=True,
               placeholder="Escolha a hora desejada",
               style={'widht':"40%"}
           ),
    dcc.Graph(
        id='crime-bar',
        figure=fig2),
])
#---------------------------------------------------------------------------
## Conectando os gráficos com os componentes do Dash
@app.callback(Output(component_id='crime-map', component_property='figure'),
     [Input(component_id='bar-selector', component_property='value')])
def update_graph(hours_selected):

  #Teste para verificar se o objeto é vazio ou nulo
  if (hours_selected is None or len(hours_selected) == 0):
    hours_selected = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

  update_crime_map = crime_map.copy()
  update_crime_map = update_crime_map[update_crime_map.aux.isin(hours_selected)]

  fig = px.scatter_mapbox(update_crime_map, 
                        lat='latitude', 
                        lon='longitude',
                        size_max=40, 
                        zoom=10,                      
                        color='aux',
                        labels={'aux':'Hora'},
                        color_continuous_scale=px.colors.sequential.Viridis_r,                          
                        mapbox_style='carto-positron')
  fig.update_layout(mapbox_style="dark", mapbox_accesstoken=token)
  fig.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
  fig.update_layout(coloraxis_colorbar=dict(
      title="Horário do dia",
      bgcolor= None,   
      dtick=1,
      tickfont=dict(color='white'),
      titlefont=dict(color='white'),
  ))
  fig.update_layout({
  'plot_bgcolor': 'rgba(0, 0, 0, 0)',
  'paper_bgcolor': 'rgba(0, 0, 0, 0)',
  })



  return fig

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Overwriting my_dash_app.py


In [ ]:
# 21 -
retorno = get_tunnel()
print(retorno)
!python my_dash_app.py

Response(url='http://c251-35-224-140-233.ngrok.io', error=None)
/content/my_dash_app.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/content/my_dash_app.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/content/my_dash_app.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime["hora"] = pd.to_datetime(crime.time.str.rsplit(n=1, expand=True)[1].str[:10])
/content/my_dash_app.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.